In [8]:
import numpy as np

In [9]:
# f = w*x
# f = 2 * x

In [11]:
def forward(x):
    return w*x 

def loss(y, y_pred):
    # MSE
    return np.mean((y - y_pred)**2)

# Gradient
# MSE J = 1/n * (w * x - y)^2
# dJ/dw = 1/n * 2x * (wx - y)
def gradient(x, y, y_pred):
    return np.mean(np.dot(2*x, y_pred - y))

In [17]:
X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)
w = 0.01

print(f'Prediction before training: f(5) = {forward(5):.3f}')
# Training
learning_rate = 0.01
n_iters = 6

for epoch in range(n_iters):
    y_pred = forward(X)
    l = loss(Y, y_pred)
    dw = gradient(X, Y, y_pred)
    w -= learning_rate * dw 

    if epoch % 2 == 0:
        print(f"Epoch {epoch+1}: w = {w:.4f}, loss = {l:.4f}")

print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.050
Epoch 1: w = 1.2040, loss = 29.7007
Epoch 3: w = 1.8726, loss = 0.7603
Epoch 5: w = 1.9796, loss = 0.0195
Prediction after training: f(5) = 9.959


In [18]:
import torch

In [24]:
def forward(x):
    return w*x 

def loss(y, y_pred):
    # MSE
    return ((y - y_pred)**2).mean()

# Gradient
# MSE J = 1/n * (w * x - y)^2
# dJ/dw = 1/n * 2x * (wx - y)

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)
w = torch.tensor(0.11, dtype=torch.float32, requires_grad=True)

print(f'Prediction before training: f(5) = {forward(5):.3f}')
# Training
learning_rate = 0.01
n_iters = 200

for epoch in range(n_iters):
    y_pred = forward(X)
    l = loss(Y, y_pred)
    #dw = gradient(X, Y, y_pred)
    l.backward()
    #w -= learning_rate * dw
    with torch.no_grad(): # This updating needs not be a part of computational graph
        w -= learning_rate * w.grad
    # Don't accumulate the grad(W) ## But W values themselves allowed to change
    w.grad.zero_()

    if epoch % 20 == 0:
        print(f"Epoch {epoch+1}: w = {w:.4f}, loss = {l:.4f}")

print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.550
Epoch 1: w = 0.3935, loss = 26.7907
Epoch 21: w = 1.9377, loss = 0.0402
Epoch 41: w = 1.9976, loss = 0.0001
Epoch 61: w = 1.9999, loss = 0.0000
Epoch 81: w = 2.0000, loss = 0.0000
Epoch 101: w = 2.0000, loss = 0.0000
Epoch 121: w = 2.0000, loss = 0.0000
Epoch 141: w = 2.0000, loss = 0.0000
Epoch 161: w = 2.0000, loss = 0.0000
Epoch 181: w = 2.0000, loss = 0.0000
Prediction after training: f(5) = 10.000


In [25]:
# General steps
# 1. Design model: input, output size, forward pass
# 2. Construct loss and optimizer
# 3. Training loop
#       - Forward pass: compute prediction
#       - Backward pass: gradients
#       - Update weights

In [26]:
import torch.nn as nn 

In [29]:
# f = w * x
# f = 2 * x

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

n_samples, n_features = X.shape

input_size = n_features
output_size = n_features 

model = nn.Linear(input_size, output_size)

loss = nn.MSELoss() 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(f"Pred before training: f(5) = {model(X_test).item():.3f}")

for epoch in range(n_iters):
    # Forward
    y_pred = model(X)
    # Loss
    l = loss(Y, y_pred)
    # Grad = backward pass
    l.backward() #dl/dw
    # Update weights
    optimizer.step() # instead of doing w -= lr * dw
    # Empty out grads
    optimizer.zero_grad()

    if epoch % 20 == 0:
        [w, b] = model.parameters()
        print(f"Epoch {epoch+1}: w = {w[0][0].item():.4f}, loss = {l:.4f}")

print(f"Pred after training: f(5) = {model(X_test).item():.3f}")
    


Pred before training: f(5) = 2.885
Epoch 1: w = 0.9021, loss = 16.7068
Epoch 21: w = 1.9877, loss = 0.0116
Epoch 41: w = 2.0149, loss = 0.0004
Epoch 61: w = 2.0147, loss = 0.0003
Epoch 81: w = 2.0138, loss = 0.0003
Epoch 101: w = 2.0130, loss = 0.0002
Epoch 121: w = 2.0123, loss = 0.0002
Epoch 141: w = 2.0116, loss = 0.0002
Epoch 161: w = 2.0109, loss = 0.0002
Epoch 181: w = 2.0103, loss = 0.0002
Pred after training: f(5) = 10.020


In [30]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # Define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)

loss = nn.MSELoss() 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(f"Pred before training: f(5) = {model(X_test).item():.3f}")

for epoch in range(n_iters):
    # Forward
    y_pred = model(X)
    # Loss
    l = loss(Y, y_pred)
    # Grad = backward pass
    l.backward() #dl/dw
    # Update weights
    optimizer.step() # instead of doing w -= lr * dw
    # Empty out grads
    optimizer.zero_grad()

    if epoch % 20 == 0:
        [w, b] = model.parameters()
        print(f"Epoch {epoch+1}: w = {w[0][0].item():.4f}, loss = {l:.4f}")

print(f"Pred after training: f(5) = {model(X_test).item():.3f}")

Pred before training: f(5) = -0.845
Epoch 1: w = 0.0148, loss = 32.6155
Epoch 21: w = 1.5519, loss = 0.2633
Epoch 41: w = 1.6148, loss = 0.2144
Epoch 61: w = 1.6382, loss = 0.1901
Epoch 81: w = 1.6593, loss = 0.1686
Epoch 101: w = 1.6791, loss = 0.1496
Epoch 121: w = 1.6978, loss = 0.1327
Epoch 141: w = 1.7154, loss = 0.1177
Epoch 161: w = 1.7319, loss = 0.1044
Epoch 181: w = 1.7475, loss = 0.0926
Pred after training: f(5) = 9.509
